# 🔶 Passo a Passo: Criando o Catálogo no Metastore do Databricks - Gold

- 1️⃣ Criar um Database no Metastore
- 2️⃣ Registrar a Tabela Delta no Metastore
- 3️⃣ Validar se a tabela foi registrada corretamente

### 🔹 1. Criar um Database no Metastore do Databricks

No Databricks, as tabelas são organizadas dentro de um Database. Vamos criar um database específico para armazenar os metadados.




In [0]:
%run "./00-Configuracao"

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS catalogo_de_dados
LOCATION 's3://mvp-brasileirao-2024/gold/catalogo_de_dados';

CREATE DATABASE IF NOT EXISTS marts_mvp_brasileirao

Leitura das credenciais de forma segura pelo Spark 🔐


Caminhos definidos:
📂 Bronze: s3a://mvp-brasileirao-2024/bronze/ | 📂 Silver: s3a://mvp-brasileirao-2024/silver/ | 📂 Gold: s3a://mvp-brasileirao-2024/gold/


✅ Credenciais configuradas com sucesso!



### 🔹 Registro das Tabela no Metastore

In [0]:
%sql
CREATE TABLE IF NOT EXISTS marts_mvp_brasileirao.classificacao_tier
USING PARQUET
LOCATION 's3a://mvp-brasileirao-2024/gold/Classificacao_Tier';

CREATE TABLE IF NOT EXISTS marts_mvp_brasileirao.desempenho_clubes
USING PARQUET
LOCATION 's3a://mvp-brasileirao-2024/gold/Mart_Desempenho_Clubes';

CREATE TABLE IF NOT EXISTS marts_mvp_brasileirao.desempenho_jogadores
USING PARQUET
LOCATION 's3a://mvp-brasileirao-2024/gold/Mart_Desempenho_Jogadores';

CREATE TABLE IF NOT EXISTS marts_mvp_brasileirao.informacao_jogadores
USING PARQUET
LOCATION 's3a://mvp-brasileirao-2024/gold/Mart_Info_Jogadores';

CREATE TABLE IF NOT EXISTS marts_mvp_brasileirao.todas_partidas
USING PARQUET
LOCATION 's3a://mvp-brasileirao-2024/gold/todas_partidas';

### 🔹 3. Validar o Registro no Metastore

Agora, vamos verificar se a tabela foi registrada corretamente.



In [0]:
%sql
SHOW TABLES IN marts_mvp_brasileirao;

database,tableName,isTemporary
marts_mvp_brasileirao,classificacao_tier,false
marts_mvp_brasileirao,desempenho_clubes,false
marts_mvp_brasileirao,desempenho_jogadores,false
marts_mvp_brasileirao,informacao_jogadores,false
marts_mvp_brasileirao,todas_partidas,false


# 🔶 Criação do Catálogo de Dados


## 🔷 classificacao_tier

### 🔹 Criando o Metadata do dataframe

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum, expr

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_gold = "marts_mvp_brasileirao.classificacao_tier"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_gold)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta e duplicatas)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Contagem de valores distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns],  # Contagem correta de nulos
    *[(count(col(c)) - countDistinct(col(c))).alias(f"{c}_duplicates") for c in df.columns]  # Contagem de valores duplicados
)

### 🔹 Criação do Dicionário de descrições

In [0]:
col_descriptions = {
    "posicao": (
        "Posição final do clube na tabela de classificação do campeonato.",
        "Renomeação da coluna para padronização.",
        "Bronze → Silver → Gold"
    ),
    "clube": (
        "Nome do clube participante do campeonato.",
        "Renomeação da coluna para padronização. Padronização dos nomes dos clubes para garantir consistência entre os datasets.",
        "Bronze → Silver → Gold"
    ),
    "pontuacao": (
        "Total de pontos acumulados pelo clube ao longo da competição.",
        "Renomeação da coluna para padronização.",
        "Bronze → Silver → Gold"
    ),
    "tier_clubes": (
        "Segmento (Tier) do clube com base na posição final (ex.: Tier S, A, B ou C).",
        "Coluna derivada com base na posição na classificação geral.",
        "Silver → Gold"
    ),
    "vitorias": (
        "Quantidade total de vitórias conquistadas pelo clube.",
        "Renomeação da coluna para padronização.",
        "Bronze → Silver → Gold"
    ),
    "empates": (
        "Quantidade total de empates registrados pelo clube.",
        "Renomeação da coluna para padronização.",
        "Bronze → Silver → Gold"
    ),
    "derrotas": (
        "Quantidade total de derrotas sofridas pelo clube.",
        "Renomeação da coluna para padronização.",
        "Bronze → Silver → Gold"
    ),
    "gols_marcados": (
        "Número total de gols marcados pelo clube durante a competição.",
        "Renomeação da coluna para padronização.",
        "Bronze → Silver → Gold"
    ),
    "avg_gols_marcados_tier": (
        "Média de gols marcados pelos clubes do mesmo Tier.",
        "Cálculo agregado por Tier a partir dos dados da tabela de classificação.",
        "Silver → Gold"
    ),
    "avg_gols_marcados_geral": (
        "Média geral de gols marcados por todos os clubes da competição.",
        "Cálculo agregado geral com base nos dados de gols marcados.",
        "Silver → Gold"
    ),
    "gols_sofridos": (
        "Número total de gols sofridos pelo clube durante a competição.",
        "Renomeação da coluna para padronização.",
        "Bronze → Silver → Gold"
    ),
    "avg_gols_sofridos_tier": (
        "Média de gols sofridos pelos clubes do mesmo Tier.",
        "Cálculo agregado por Tier a partir dos dados da tabela de classificação.",
        "Silver → Gold"
    ),
    "avg_gols_sofridos_geral": (
        "Média geral de gols sofridos por todos os clubes da competição.",
        "Cálculo agregado geral com base nos dados de gols sofridos.",
        "Silver → Gold"
    )
}


### 🔹 Criando o Dataframe do Catálogo

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"
    col_duplicates = metadata_df.collect()[0][f"{col_name}_duplicates"] if f"{col_name}_duplicates" in metadata_df.columns else 0  # Contagem de duplicados

    # Buscar definição, regra de transformação e linhagem dos dados no dicionário
    col_definition, transformations_applied, lineage_info = col_descriptions.get(
        col_name, (f"Descrição da coluna {col_name}", "Nenhuma alteração aplicada.", "Origem desconhecida")
    )

    # Definir observações sobre a coluna (pode ser preenchido manualmente se necessário)
    comments = "Sem observações adicionais."

    # Adicionar à lista
    catalog_data.append(Row(ID_Coluna=index,
                            Coluna=col_name, 
                            Tipo_de_Dado=col_type, 
                            Definição=col_definition,
                            Valores_Min_Max=f"{col_min} | {col_max}", 
                            Qtd_Valores_Registrados=col_count,
                            Qtd_Valores_Nulos=col_nulls,
                            Qtd_Valores_Distintos=col_distinct,
                            Qtd_Valores_Duplicados=col_duplicates,
                            Regras_de_Transformação=transformations_applied,
                            Linhagem_Dados=lineage_info,  # ✅ Agora vem direto do dicionário!
                            Observacoes=comments,
                            Fonte="Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil",
                            Link_Fonte="https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/",
                            Arquivo_Original="classificacao_brasileirao_2024.csv",
                            Arquivo_S3="s3://mvp-brasileirao-2024/Gold/Classificacao_Tier/"
                            ))                            

# Criar o DataFrame do Catálogo de Dados
catalog_df = spark.createDataFrame(catalog_data)


### 🔹 Salvando o Catálogo no S3 camada Gold

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/gold/catalogo_de_dados/classificacao_tier"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


### 🔹 Criando Tabela de Catálogos no Hive

In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_classificacao_tier;

CREATE TABLE catalogo_de_dados.catalog_classificacao_tier
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/gold/catalogo_de_dados/classificacao_tier';

### 🔹 Exibindo o Catálogo de Dados via SQL

In [0]:
%sql
select * from catalogo_de_dados.catalog_classificacao_tier

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Qtd_Valores_Duplicados,Regras_de_Transformação,Linhagem_Dados,Observacoes,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
12,avg_gols_sofridos_tier,double,Média de gols sofridos pelos clubes do mesmo Tier.,35.8 | 52.4,20,0,4,16,Cálculo agregado por Tier a partir dos dados da tabela de classificação.,Silver → Gold,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,s3://mvp-brasileirao-2024/Gold/Classificacao_Tier/
13,avg_gols_sofridos_geral,double,Média geral de gols sofridos por todos os clubes da competição.,46.45 | 46.45,20,0,1,19,Cálculo agregado geral com base nos dados de gols sofridos.,Silver → Gold,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,s3://mvp-brasileirao-2024/Gold/Classificacao_Tier/
2,clube,string,Nome do clube participante do campeonato.,Athletico-PR | Vasco da Gama,20,0,20,0,Renomeação da coluna para padronização. Padronização dos nomes dos clubes para garantir consistência entre os datasets.,Bronze → Silver → Gold,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,s3://mvp-brasileirao-2024/Gold/Classificacao_Tier/
3,pontuacao,int,Total de pontos acumulados pelo clube ao longo da competição.,30 | 79,20,0,17,3,Renomeação da coluna para padronização.,Bronze → Silver → Gold,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,s3://mvp-brasileirao-2024/Gold/Classificacao_Tier/
4,tier_clubes,string,"Segmento (Tier) do clube com base na posição final (ex.: Tier S, A, B ou C).",TIER A | TIER S,20,0,4,16,Coluna derivada com base na posição na classificação geral.,Silver → Gold,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,s3://mvp-brasileirao-2024/Gold/Classificacao_Tier/
7,derrotas,int,Quantidade total de derrotas sofridas pelo clube.,5 | 22,20,0,12,8,Renomeação da coluna para padronização.,Bronze → Silver → Gold,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,s3://mvp-brasileirao-2024/Gold/Classificacao_Tier/
8,gols_marcados,int,Número total de gols marcados pelo clube durante a competição.,29 | 61,20,0,15,5,Renomeação da coluna para padronização.,Bronze → Silver → Gold,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,s3://mvp-brasileirao-2024/Gold/Classificacao_Tier/
10,avg_gols_marcados_geral,double,Média geral de gols marcados por todos os clubes da competição.,46.45 | 46.45,20,0,1,19,Cálculo agregado geral com base nos dados de gols marcados.,Silver → Gold,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/fu

## 🔷 desempenho_clubes

### 🔹 Criando o Metadata do dataframe

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum, expr

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_gold = "marts_mvp_brasileirao.desempenho_clubes"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_gold)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta e duplicatas)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Contagem de valores distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns],  # Contagem correta de nulos
    *[(count(col(c)) - countDistinct(col(c))).alias(f"{c}_duplicates") for c in df.columns]  # Contagem de valores duplicados
)

### 🔹 Criação do Dicionário de descrições

In [0]:
col_descriptions = {
    "posicao": (
        "Posição final do clube na classificação geral.",
        "Derivado da tabela de classificação.",
        "classificacao_tier"
    ),
    "clube": (
        "Nome do clube.",
        "Derivado da tabela de classificação.",
        "classificacao_tier"
    ),
    "tier_clubes": (
        "Tierlist do clube com base na sua posição (S, A, B, C).",
        "Derivado com regras de segmentação por posição.",
        "classificacao_tier"
    ),
    "norm_gols_sofridos": (
        "Total de gols sofridos pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "classificacao_tier → Mart_Desempenho_Defensivo_Clubes → Gold"
    ),
    "norm_desarmes": (
        "Total de desarmes realizados pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Defensivo_Clubes → Gold"
    ),
    "norm_interceptacoes": (
        "Total de interceptações feitas pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Defensivo_Clubes → Gold"
    ),
    "norm_bloqueios": (
        "Total de bloqueios realizados pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Defensivo_Clubes → Gold"
    ),
    "norm_gols_marcados": (
        "Total de gols marcados pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_assistencias": (
        "Total de assistências feitas pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_penaltis_batidos": (
        "Total de pênaltis batidos pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_total_chutes": (
        "Total de chutes realizados pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_chutes_gol": (
        "Total de chutes no alvo do clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_gols_esperados": (
        "Total de gols esperados (xG) do clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_ass_esperadas": (
        "Total de assistências esperadas (xA) do clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_acoes_de_criacao": (
        "Total de ações de criação de jogadas do clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_acoes_de_gol": (
        "Total de ações que levaram a gol, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold"
    ),
    "norm_eficiencia": (
        "Índice de eficiência geral ofensiva e defensiva do clube, normalizado com Min-Max Scaling (0 a 10).",
        "Média ponderada de métricas-chave, normalizada por Min-Max com ajuste para escala de 0 a 10.",
        "cálculo próprio"
    ),
    "norm_passes_completos": (
        "Total de passes completos do clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Criacao_Clubes → Gold"
    ),
    "norm_passes_tentados": (
        "Total de passes tentados pelo clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Criacao_Clubes → Gold"
    ),
    "norm_conducoes_de_bola": (
        "Total de conduções de bola do clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Criacao_Clubes → Gold"
    ),
    "norm_conducoes_progressivas": (
        "Total de conduções progressivas de bola do clube, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado",
        "Estatisticas_PorClube → Mart_Desempenho_Criacao_Clubes → Gold"
    ),
    "score_defensivo": (
        "Score defensivo agregado do clube.",
        "Pontuação calculada com base nas métricas defensivas normalizadas (desarmes, interceptações, bloqueios e gols sofridos), ponderadas e combinadas.",
        "cálculo próprio"
    ),
    "score_ofensivo": (
        "Score ofensivo agregado do clube.",
        "Pontuação calculada com base nas métricas ofensivas normalizadas (gols marcados, assistências, chutes, xG, etc.), ponderadas e combinadas.",
        "cálculo próprio"
    ),
    "score_criacao": (
        "Score de criação de jogadas do clube.",
        "Pontuação baseada em métricas de criação (ações de criação, passes esperados, assistências, etc.), normalizadas e combinadas.",
        "cálculo próprio"
)
}


### 🔹 Criando o Dataframe do Catálogo

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"
    col_duplicates = metadata_df.collect()[0][f"{col_name}_duplicates"] if f"{col_name}_duplicates" in metadata_df.columns else 0  # Contagem de duplicados

    # Buscar definição, regra de transformação e linhagem dos dados no dicionário
    col_definition, transformations_applied, lineage_info = col_descriptions.get(
        col_name, (f"Descrição da coluna {col_name}", "Nenhuma alteração aplicada.", "Origem desconhecida")
    )

    # Definir observações sobre a coluna (pode ser preenchido manualmente se necessário)
    comments = "Sem observações adicionais."

    catalog_data.append(Row(
        ID_Coluna=index,
        Coluna=col_name,
        Tipo_de_Dado=col_type,
        Definição=col_definition,
        Valores_Min_Max=f"{col_min} | {col_max}",
        Qtd_Valores_Registrados=col_count,
        Qtd_Valores_Nulos=col_nulls,
        Qtd_Valores_Distintos=col_distinct,
        Qtd_Valores_Duplicados=col_duplicates,
        Regras_de_Transformação=transformations_applied,
        Linhagem_Dados=lineage_info,
        Observacoes=comments,
        Fonte="Tabela construída a partir da agregação de estatísticas individuais dos jogadores por clube.",
        Link_Fonte="https://fbref.com/",
        Arquivo_Original="BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv",
        Arquivo_S3="s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Clubes/"
    ))

catalog_df = spark.createDataFrame(catalog_data)


### 🔹 Salvando o Catálogo no S3 camada Gold

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/gold/catalogo_de_dados/desempenho_clubes"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


### 🔹 Criando Tabela de Catálogos no Hive

In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_desempenho_clubes;

CREATE TABLE catalogo_de_dados.catalog_desempenho_clubes
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/gold/catalogo_de_dados/desempenho_clubes';

### 🔹 Exibindo o Catálogo de Dados via SQL

In [0]:
%sql
select * from catalogo_de_dados.catalog_desempenho_clubes

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Qtd_Valores_Duplicados,Regras_de_Transformação,Linhagem_Dados,Observacoes,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
13,norm_gols_esperados,double,"Total de gols esperados (xG) do clube, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,20,0,20,0,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado,Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold,Sem observações adicionais.,Tabela construída a partir da agregação de estatísticas individuais dos jogadores por clube.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Clubes/
14,norm_ass_esperadas,double,"Total de assistências esperadas (xA) do clube, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,20,0,20,0,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado,Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold,Sem observações adicionais.,Tabela construída a partir da agregação de estatísticas individuais dos jogadores por clube.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Clubes/
15,norm_acoes_de_criacao,double,"Total de ações de criação de jogadas do clube, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,20,0,20,0,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado,Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold,Sem observações adicionais.,Tabela construída a partir da agregação de estatísticas individuais dos jogadores por clube.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Clubes/
19,norm_passes_tentados,double,"Total de passes tentados pelo clube, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,20,0,20,0,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado,Estatisticas_PorClube → Mart_Desempenho_Criacao_Clubes → Gold,Sem observações adicionais.,Tabela construída a partir da agregação de estatísticas individuais dos jogadores por clube.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Clubes/
20,norm_conducoes_de_bola,double,"Total de conduções de bola do clube, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,20,0,20,0,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado,Estatisticas_PorClube → Mart_Desempenho_Criacao_Clubes → Gold,Sem observações adicionais.,Tabela construída a partir da agregação de estatísticas individuais dos jogadores por clube.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Clubes/
21,norm_conducoes_progressivas,double,"Total de conduções progressivas de bola do clube, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,20,0,20,0,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado,Estatisticas_PorClube → Mart_Desempenho_Criacao_Clubes → Gold,Sem observações adicionais.,Tabela construída a partir da agregação de estatísticas individuais dos jogadores por clube.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Clubes/
10,norm_penaltis_batidos,double,"Total de pênaltis batidos pelo clube, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,20,0,10,10,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado,Estatisticas_PorClube → Mart_Desempenho_Ofensivo_Clubes → Gold,Sem observações adicionais.,Tabela construída a partir da agregação de estatísticas individuais dos jogadores por clube.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Clubes/
11,norm_total_chutes,double,"Total de chutes realizados pelo clube, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,20,0,19,1,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado,Estatisticas_PorClube → Mart_Desempen

## 🔷 desempenho_jogadores

### 🔹 Criando o Metadata do dataframe

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum, expr

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_gold = "marts_mvp_brasileirao.desempenho_jogadores"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_gold)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta e duplicatas)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Contagem de valores distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns],  # Contagem correta de nulos
    *[(count(col(c)) - countDistinct(col(c))).alias(f"{c}_duplicates") for c in df.columns]  # Contagem de valores duplicados
)

### 🔹 Criação do Dicionário de descrições

In [0]:
col_descriptions = {
    "jogador": (
        "Nome do jogador.",
        "Derivado da tabela de Estatisticas_PorJogador.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "clube": (
        "Nome do clube ao qual o jogador pertence.",
        "Derivado da tabela de Estatisticas_PorJogador.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "numero_camisa": (
        "Número da camisa do jogador.",
        "Derivado da tabela de Estatisticas_PorJogador.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "minutos_em_campo": (
        "Total de minutos que o jogador esteve em campo ao longo da competição.",
        "Derivado da tabela de Estatisticas_PorJogador.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_gols_sofridos_clube": (
        "Gols sofridos pelo clube enquanto o jogador estava em campo, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_desarmes": (
        "Total de desarmes realizados pelo jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_interceptacoes": (
        "Total de interceptações feitas pelo jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_bloqueios": (
        "Total de bloqueios realizados pelo jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_gols_marcados": (
        "Total de gols marcados pelo jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_ass": (
        "Total de assistências do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_penaltis_batidos": (
        "Total de pênaltis batidos pelo jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_total_chutes": (
        "Total de chutes do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_chutes_gol": (
        "Total de chutes no alvo do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_gols_esperados": (
        "xG (gols esperados) do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_ass_esperadas": (
        "xA (assistências esperadas) do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_acoes_de_criacao": (
        "Total de ações de criação de jogadas do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_acoes_de_gol": (
        "Total de ações diretamente ligadas a gols do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_eficiencia": (
        "Índice geral de eficiência do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "cálculo próprio"
    ),
    "norm_passes_completos": (
        "Total de passes completos do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_passes_tentados": (
        "Total de passes tentados pelo jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_conducoes_de_bola": (
        "Total de conduções de bola do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "norm_conducoes_progressivas": (
        "Conduções progressivas (em direção ao ataque) do jogador, normalizado com Min-Max Scaling (0 a 10).",
        "Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "score_defensivo": (
        "Score defensivo agregado do jogador.",
        "Pontuação calculada com base nas métricas defensivas normalizadas (desarmes, interceptações, bloqueios e gols sofridos), ponderadas e combinadas.",
        "cálculo próprio"
    ),
    "score_ofensivo": (
        "Score ofensivo agregado do jogador.",
        "Pontuação calculada com base nas métricas ofensivas normalizadas (gols marcados, assistências, chutes, xG, etc.), ponderadas e combinadas.",
        "cálculo próprio"
    ),
    "score_criacao": (
        "Score de criação de jogadas do jogador.",
        "Pontuação baseada em métricas de criação (ações de criação, passes esperados, assistências, etc.), normalizadas e combinadas.",
        "cálculo próprio"
    )
}


### 🔹 Criando o Dataframe do Catálogo

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"
    col_duplicates = metadata_df.collect()[0][f"{col_name}_duplicates"] if f"{col_name}_duplicates" in metadata_df.columns else 0  # Contagem de duplicados

    # Buscar definição, regra de transformação e linhagem dos dados no dicionário
    col_definition, transformations_applied, lineage_info = col_descriptions.get(
        col_name, (f"Descrição da coluna {col_name}", "Nenhuma alteração aplicada.", "Origem desconhecida")
    )

    # Definir observações sobre a coluna (pode ser preenchido manualmente se necessário)
    comments = "Sem observações adicionais."

    catalog_data.append(Row(
        ID_Coluna=index,
        Coluna=col_name,
        Tipo_de_Dado=col_type,
        Definição=col_definition,
        Valores_Min_Max=f"{col_min} | {col_max}",
        Qtd_Valores_Registrados=col_count,
        Qtd_Valores_Nulos=col_nulls,
        Qtd_Valores_Distintos=col_distinct,
        Qtd_Valores_Duplicados=col_duplicates,
        Regras_de_Transformação=transformations_applied,
        Linhagem_Dados=lineage_info,
        Observacoes=comments,
        Fonte="Tabela construída a partir da extração e processamento das estatísticas individuais dos jogadores.",
        Link_Fonte="https://fbref.com/",
        Arquivo_Original="BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv",
        Arquivo_S3="s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Jogadores/"
    ))

catalog_df = spark.createDataFrame(catalog_data)

### 🔹 Salvando o Catálogo no S3 camada Gold

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/gold/catalogo_de_dados/desempenho_jogadores"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


### 🔹 Criando Tabela de Catálogos no Hive

In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_desempenho_jogadores;

CREATE TABLE catalogo_de_dados.catalog_desempenho_jogadores
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/gold/catalogo_de_dados/desempenho_jogadores';

### 🔹 Exibindo o Catálogo de Dados via SQL

In [0]:
%sql
select * from catalogo_de_dados.catalog_desempenho_jogadores

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Qtd_Valores_Duplicados,Regras_de_Transformação,Linhagem_Dados,Observacoes,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
22,norm_conducoes_progressivas,double,"Conduções progressivas (em direção ao ataque) do jogador, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,838,0,83,755,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.,Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da extração e processamento das estatísticas individuais dos jogadores.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Jogadores/
23,score_defensivo,double,Score defensivo agregado do jogador.,0.0 | 9.46,838,0,430,408,"Pontuação calculada com base nas métricas defensivas normalizadas (desarmes, interceptações, bloqueios e gols sofridos), ponderadas e combinadas.",cálculo próprio,Sem observações adicionais.,Tabela construída a partir da extração e processamento das estatísticas individuais dos jogadores.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Jogadores/
24,score_ofensivo,double,Score ofensivo agregado do jogador.,0.01 | 7.81,582,256,255,327,"Pontuação calculada com base nas métricas ofensivas normalizadas (gols marcados, assistências, chutes, xG, etc.), ponderadas e combinadas.",cálculo próprio,Sem observações adicionais.,Tabela construída a partir da extração e processamento das estatísticas individuais dos jogadores.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Jogadores/
25,score_criacao,double,Score de criação de jogadas do jogador.,0.0 | 9.19,838,0,283,555,"Pontuação baseada em métricas de criação (ações de criação, passes esperados, assistências, etc.), normalizadas e combinadas.",cálculo próprio,Sem observações adicionais.,Tabela construída a partir da extração e processamento das estatísticas individuais dos jogadores.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Jogadores/
19,norm_passes_completos,double,"Total de passes completos do jogador, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,838,0,351,487,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.,Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da extração e processamento das estatísticas individuais dos jogadores.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Jogadores/
20,norm_passes_tentados,double,"Total de passes tentados pelo jogador, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,838,0,381,457,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.,Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da extração e processamento das estatísticas individuais dos jogadores.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Jogadores/
21,norm_conducoes_de_bola,double,"Total de conduções de bola do jogador, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,838,0,360,478,Cálculo Normalização Min-Max (escala de 0 a 10) aplicado.,Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da extração e processamento das estatísticas individuais dos jogadores.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Desempenho_Jogadores/
7,norm_interceptacoes,double,"Total de interceptações feitas pelo jogador, normalizado com Min-Max Scaling (0 a 10).",0.0 | 10.0,838,0,48,790,Cálculo Normalização Min-Max (escala de 0 a 10) apli

## 🔷 informacao_jogadores

### 🔹 Criando o Metadata do dataframe

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum, expr

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_gold = "marts_mvp_brasileirao.informacao_jogadores"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_gold)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta e duplicatas)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Contagem de valores distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns],  # Contagem correta de nulos
    *[(count(col(c)) - countDistinct(col(c))).alias(f"{c}_duplicates") for c in df.columns]  # Contagem de valores duplicados
)

### 🔹 Criação do Dicionário de descrições

In [0]:
col_descriptions = {
    "Jogador": (
        "Nome do jogador.",
        "Derivado da tabela de Estatisticas_PorJogador.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "Clube": (
        "Clube que o jogador representou no Campeonato Brasileiro de 2024.",
        "Derivado da tabela de Estatisticas_PorJogador.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "Nacionalidade": (
        "País de origem do jogador.",
        "Derivado da tabela de Estatisticas_PorJogador.",
        "Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold"
    ),
    "Posicao_Principal": (
        "Posição em que o jogador atuou com maior frequência na temporada.",
        "Calculado a partir das posições registradas na Estatisticas_PorJogador.",
        "Estatisticas_PorJogador → Mart_Info_Jogadores → Gold"
    ),
    "Nome_Posicao_Principal": (
        "Nome da posição principal que o jogador jogou na partida.",
        "Derivado da coluna `posicao_principal`, com tradução das siglas para facilitar a leitura e interpretação.",
        "Estatisticas_PorJogador → Mart_Info_Jogadores → Gold"
    ),    
    "jogos_Posicao_Principal": (
        "Quantidade de partidas em que o jogador atuou na sua posição principal.",
        "Calculado por meio da contagem das partidas por posição.",
        "Estatisticas_PorJogador → Mart_Info_Jogadores → Gold"
    ),
    "Improvisacao": (
        "Outra posição em que o jogador foi escalado, diferente da sua função original.",
        "Derivado da análise de múltiplas posições por jogador.",
        "Estatisticas_PorJogador → Mart_Info_Jogadores → Gold"
    ),
    "Nome_Improvisacao": (
        "Nome da posição em que o jogador atuou improvisado durante a partida",
        "Derivado da coluna `improvisacao`, com tradução das siglas de posição para facilitar a análise.",
        "Estatisticas_PorJogador → Mart_Info_Jogadores → Gold"
    ),    
    "jogos_improvisados": (
        "Quantidade de partidas em que o jogador foi escalado fora da sua posição principal.",
        "Contagem de jogos em posições diferentes da principal.",
        "Estatisticas_PorJogador → Mart_Info_Jogadores → Gold"
    )
}


### 🔹 Criando o Dataframe do Catálogo

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"
    col_duplicates = metadata_df.collect()[0][f"{col_name}_duplicates"] if f"{col_name}_duplicates" in metadata_df.columns else 0  # Contagem de duplicados

    # Buscar definição, regra de transformação e linhagem dos dados no dicionário
    col_definition, transformations_applied, lineage_info = col_descriptions.get(
        col_name, (f"Descrição da coluna {col_name}", "Nenhuma alteração aplicada.", "Origem desconhecida")
    )

    # Definir observações sobre a coluna (pode ser preenchido manualmente se necessário)
    comments = "Sem observações adicionais."

    catalog_data.append(Row(
        ID_Coluna=index,
        Coluna=col_name,
        Tipo_de_Dado=col_type,
        Definição=col_definition,
        Valores_Min_Max=f"{col_min} | {col_max}",
        Qtd_Valores_Registrados=col_count,
        Qtd_Valores_Nulos=col_nulls,
        Qtd_Valores_Distintos=col_distinct,
        Qtd_Valores_Duplicados=col_duplicates,
        Regras_de_Transformação=transformations_applied,
        Linhagem_Dados=lineage_info,
        Observacoes=comments,
        Fonte = "Tabela construída a partir da análise da posição principal e das improvisações dos jogadores ao longo da temporada.",
        Link_Fonte = "https://fbref.com/",
        Arquivo_Original = "BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv",
        Arquivo_S3 = "s3://mvp-brasileirao-2024/Gold/Mart_Info_Jogadores/"

    ))

catalog_df = spark.createDataFrame(catalog_data)

### 🔹 Salvando o Catálogo no S3 camada Gold

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/gold/catalogo_de_dados/informacao_jogadores"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


### 🔹 Criando Tabela de Catálogos no Hive

In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_informacao_jogadores;

CREATE TABLE catalogo_de_dados.catalog_informacao_jogadores
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/gold/catalogo_de_dados/informacao_jogadores';

### 🔹 Exibindo o Catálogo de Dados via SQL

In [0]:
%sql
select * from catalogo_de_dados.catalog_informacao_jogadores

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Qtd_Valores_Duplicados,Regras_de_Transformação,Linhagem_Dados,Observacoes,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
5,Nome_Posicao_Principal,string,Nome da posição principal que o jogador jogou na partida.,Ala | Zagueiro,726,0,13,713,"Derivado da coluna `posicao_principal`, com tradução das siglas para facilitar a leitura e interpretação.",Estatisticas_PorJogador → Mart_Info_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da análise da posição principal e das improvisações dos jogadores ao longo da temporada.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Info_Jogadores/
8,Nome_Improvisacao,string,Nome da posição em que o jogador atuou improvisado durante a partida,Ala | Zagueiro,569,157,12,557,"Derivado da coluna `improvisacao`, com tradução das siglas de posição para facilitar a análise.",Estatisticas_PorJogador → Mart_Info_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da análise da posição principal e das improvisações dos jogadores ao longo da temporada.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Info_Jogadores/
9,jogos_improvisados,bigint,Quantidade de partidas em que o jogador foi escalado fora da sua posição principal.,1 | 18,569,157,16,553,Contagem de jogos em posições diferentes da principal.,Estatisticas_PorJogador → Mart_Info_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da análise da posição principal e das improvisações dos jogadores ao longo da temporada.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Info_Jogadores/
4,Posicao_Principal,string,Posição em que o jogador atuou com maior frequência na temporada.,AM | WB,726,0,13,713,Calculado a partir das posições registradas na Estatisticas_PorJogador.,Estatisticas_PorJogador → Mart_Info_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da análise da posição principal e das improvisações dos jogadores ao longo da temporada.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Info_Jogadores/
7,Improvisacao,string,"Outra posição em que o jogador foi escalado, diferente da sua função original.",AM | WB,569,157,12,557,Derivado da análise de múltiplas posições por jogador.,Estatisticas_PorJogador → Mart_Info_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da análise da posição principal e das improvisações dos jogadores ao longo da temporada.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Info_Jogadores/
2,Clube,string,Clube que o jogador representou no Campeonato Brasileiro de 2024.,Athletico-PR | Vasco da Gama,726,0,20,706,Derivado da tabela de Estatisticas_PorJogador.,Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da análise da posição principal e das improvisações dos jogadores ao longo da temporada.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Info_Jogadores/
3,Nacionalidade,string,País de origem do jogador.,ANG | VEN,726,0,24,702,Derivado da tabela de Estatisticas_PorJogador.,Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold,Sem observações adicionais.,Tabela construída a partir da análise da posição principal e das improvisações dos jogadores ao longo da temporada.,https://fbref.com/,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv,s3://mvp-brasileirao-2024/Gold/Mart_Info_Jogadores/
1,Jogador,string,Nome do jogador.,Abner | Óscar Romero,726,0,653,73,Derivado da tabela de Estatisticas_PorJogador.,Estatisticas_PorJogador → Mart_Desempenho_Jogadores → Gold,Sem observações adicionais.,Tabela construída 

## 🔷 todas_partidas

### 🔹 Criando o Metadata do dataframe

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum, expr

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_gold = "marts_mvp_brasileirao.todas_partidas"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_gold)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta e duplicatas)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Contagem de valores distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns],  # Contagem correta de nulos
    *[(count(col(c)) - countDistinct(col(c))).alias(f"{c}_duplicates") for c in df.columns]  # Contagem de valores duplicados
)

### 🔹 Criação do Dicionário de descrições

In [0]:
col_descriptions = {
    # Colunas originais da Bronze (renomeadas na Silver)
    "pais": ("País onde o campeonato foi disputado", "Renomeação da coluna para padronização", "Bronze → Silver → Gold"),
    "liga": ("Nome da liga do campeonato", "Renomeação da coluna para padronização", "Bronze → Silver → Gold"),
    "temporada": ("Ano da temporada do campeonato", "Renomeação da coluna para padronização | Filtragem para manter apenas 2024", "Bronze → Silver → Gold"),
    
    # 🔹 Atualização da coluna "Data"
    "data": ("Data em que a partida foi disputada", 
             "Renomeação da coluna e conversão para formato de data | Ajuste das datas das partidas não encontradas na base Estatistica por Jogador (-1 dia, quando necessário)", 
             "Bronze → Silver → Gold"),
    
    "horario": ("Horário da partida", "Renomeação da coluna para padronização", "Bronze → Silver → Gold"),
    
    # Novas colunas criadas na Silver
    "match_id": ("Identificador único da partida", "Criado concatenando 'Match_' com um ID incremental", "Silver (Coluna nova) → Gold"),
    "trimestre": ("Trimestre em que a partida foi disputada", "Criado a partir da data do jogo", "Silver (Coluna nova) → Gold"),
    "turno": ("Indicação se a partida foi no primeiro ou segundo turno do campeonato", "Criado com base na data do jogo", "Silver (Coluna nova) → Gold"),
    
    # Transformações para análise individual de cada time
    "clube": ("Nome do time participante da partida (mandante ou visitante)", "Criado a partir das colunas Home e Away, e unificando em uma única coluna", "Silver (Transformação) → Gold"),
    "mandantevisitante": ("Indica se o time jogou em casa ('Mandante') ou fora ('Visitante')", "Transformado a partir dos DataFrames 'df_home' e 'df_away' e criado a partir da união de ambos", "Silver (Transformação) → Gold"),
    "goals": ("Número de gols marcados pelo time na partida", "Criado unificando HG (gols do mandante) e AG (gols do visitante)", "Silver (Coluna nova) → Gold"),
    
    # Transformações aplicadas à coluna Resultado
    "resultado": ("Resultado da partida do ponto de vista do time (Vitória, Derrota ou Empate)", "Ajustado conforme a posição do time na partida", "Silver (Transformação) → Gold"),
    
    # 🔹 Nova coluna "Status_Ajuste"
    "status_ajustes": ("Indica se a data da partida foi ajustada", 
                      "Criado para identificar alterações na data das partidas. Valores possíveis: 'Ajustado -1 dia' ou 'Sem modificação'", 
                      "Silver (Coluna nova) → Gold")
}


### 🔹 Criação do Dicionário de observações

In [0]:
observacoes = {
    "data": "⚠️ Ajuste manual: Datas ajustadas para garantir a correspondência entre os arquivos de partidas e estatísticas. Quando necessário, foi subtraído 1 dia das partidas não encontradas na estatística."
}

### 🔹 Criando o Dataframe do Catálogo

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"
    col_duplicates = metadata_df.collect()[0][f"{col_name}_duplicates"] if f"{col_name}_duplicates" in metadata_df.columns else 0  # Contagem de duplicados

    # Buscar definição, regra de transformação e linhagem dos dados no dicionário
    col_definition, transformations_applied, lineage_info = col_descriptions.get(
        col_name, (f"Descrição da coluna {col_name}", "Nenhuma alteração aplicada.", "Origem desconhecida")
    )

    # Definir observações sobre a coluna (pode ser preenchido manualmente se necessário)
    comments = observacoes.get(col_name, "Sem observações adicionais.")

    # Adicionar à lista
    catalog_data.append(Row(ID_Coluna=index,
                            Coluna=col_name, 
                            Tipo_de_Dado=col_type, 
                            Definição=col_definition,
                            Valores_Min_Max=f"{col_min} | {col_max}", 
                            Qtd_Valores_Registrados=col_count,
                            Qtd_Valores_Nulos=col_nulls,
                            Qtd_Valores_Distintos=col_distinct,
                            Qtd_Valores_Duplicados=col_duplicates,
                            Regras_de_Transformação=transformations_applied,
                            Linhagem_Dados=lineage_info,  # ✅ Agora vem direto do dicionário!
                            Observacoes=comments,
                            Fonte="Kaggle",
                            Link_Fonte="https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship",
                            Arquivo_Original="BRA.csv",
                            Arquivo_S3="s3://mvp-brasileirao-2024/Gold/todas_partidas/"
                            ))                            

# Criar o DataFrame do Catálogo de Dados
catalog_df = spark.createDataFrame(catalog_data)

### 🔹 Salvando o Catálogo no S3 camada Gold

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/gold/catalogo_de_dados/todas_partidas"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


### 🔹 Criando Tabela de Catálogos no Hive

In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_todas_partidas;

CREATE TABLE catalogo_de_dados.catalog_todas_partidas
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/gold/catalogo_de_dados/todas_partidas';

### 🔹 Exibindo o Catálogo de Dados via SQL

In [0]:
%sql
select * from catalogo_de_dados.catalog_todas_partidas

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Qtd_Valores_Duplicados,Regras_de_Transformação,Linhagem_Dados,Observacoes,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
7,Clube,string,Descrição da coluna Clube,Athletico-PR | Vasco da Gama,760,0,20,740,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
8,MandanteVisitante,string,Descrição da coluna MandanteVisitante,Mandante | Visitante,760,0,2,758,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
12,Data,date,Descrição da coluna Data,2024-04-13 | 2024-12-08,760,0,110,650,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
13,Status_Ajustes,string,Descrição da coluna Status_Ajustes,Sem modificação | Sem modificação,760,0,1,759,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
10,Resultado,string,Descrição da coluna Resultado,Derrota | Vitoria,760,0,3,757,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
11,Match_ID,string,Descrição da coluna Match_ID,Match_0 | Match_99,760,0,380,380,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
2,Liga,string,Descrição da coluna Liga,Serie A | Serie A,760,0,1,759,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
3,Temporada,int,Descrição da coluna Temporada,2024 | 2024,760,0,1,759,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
5,Trimestre,int,Descrição da coluna Trimestre,2 | 4,760,0,3,757,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
6,Turno,string,Descrição da coluna Turno,Primeiro Turno | Segundo Turno,760,0,2,758,Nenhuma alteração aplicada.,Origem desconhecida,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,s3://mvp-brasileirao-2024/Gold/todas_partidas/
